# 对象的创作与毁坏

> **上级页面**: ["对象"]("/model/object/")

---



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 设置 base_url 以使输出更清晰
model.factory.base_url = 'http://test.com/museum/'




## 简介

本节涵盖对象存在的开始与结束，以及艺术家与对象创作的归属关系。

## 基础创作活动

对象生命周期中的第一个活动是其创作，使用 `Production` 类来描述。从对象到活动的关系是 `produced_by`，而 `Production` 活动本身遵循通用的[基础活动模型](/model/base/#events-and-activities)，包括时间、地点和行动者的描述。

**示例**：

与基本模式中的制作示例类似，《夜巡》由伦勃朗于1642年在阿姆斯特丹创作。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( Night Watch painting)
# 行为者: 伦勃朗 (the artist)
# 对象: HumanMadeObject representing "《夜巡》" painting
# 目的: To document the artwork and its production information
night_watch = model.HumanMadeObject(ident="nightwatch/5", label="Night Watch by Rembrandt")

# 步骤 4: 创建相关对象和关系
# Production event - when and how the painting was created
production = model.Production()
night_watch.produced_by = production

# Who carried out the production: Rembrandt (the artist)
production.carried_out_by = model.Person(ident="rembrandt", label="Rembrandt")

# When: Time period of creation (1642)
timespan = model.TimeSpan(label="1642")
timespan.begin_of_the_begin = "1642-01-01T00:00:00Z"
timespan.end_of_the_end = "1642-12-31T23:59:59Z"
production.timespan = timespan

# Where: Location of creation (Amsterdam)
production.took_place_at = model.Place(ident="amsterdam", label="Amsterdam")

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(night_watch, compact=False))



## 技法与分类

我们区分用于创作艺术品的技法和其它分类。技法有自己的属性（`technique` 而不是 `classified_as`），因此可以很容易地与其它分类区分开来。

### 技法

如果已知在对象创作中使用了特定技法，可以使用 `technique` 属性来表达，引用该技法的受控词表术语。这应该用于捕捉特定技法或方法，而基础的 `classified_as` 属性用于活动的更一般分类。更一般的分类在单个角色中更常见，将在下面讨论。

**示例**：

[雕塑《男子半身像》](https://collections.britishart.yale.edu/catalog/tms:54430)由弗朗西斯·哈德伍德使用雕塑技法创作。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (t of a Man sculpture)
# 行为者: Francis Harwood (the sculptor)
# 对象: Sculpture object representing "Bust of a Man"
# 目的: To document the sculpture and its production technique
sculpture = vocab.Sculpture(ident="bust/1", label="Bust of a Man")

# 步骤 4: 创建相关对象和关系
# Production event
production = model.Production()
sculpture.produced_by = production

# Who carried out the production: Francis Harwood (the sculptor)
production.carried_out_by = model.Person(
    ident="http://vocab.getty.edu/ulan/500015886",
    label="Francis Harwood"
)

# What technique was used: Sculpting
production.technique = vocab.instances['sculpting']

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))



## 多位艺术家（不同角色）

如果多位艺术家合作创作同一件艺术品，我们遵循分割模式，创建主 `Production` 活动的独立部分。每个组成部分捕捉特定艺术家在对象创作中角色的详细信息。这使我们能够为每位艺术家的贡献声明不同的属性，包括不同的时间、技法、角色、地点或影响。

为了保持一致性，建议在只知道一位艺术家的情况下也使用此模式，以便更容易地为作品添加更多贡献者而无需重构内容。然而，这不是必需的，以确保与现有系统的兼容性。

请注意，这些制作部分也可以有与之关联的断言，除了分类、技法或其他结构化数据之外，更详细地描述角色。

**示例**：

一幅名为《RÜN》的[彩绘纺织品](https://artgallery.yale.edu/collections/objects/153900)，其中亚麻布由莎拉·帕克手工编织，然后由马克·巴罗绘画。




In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( painted textile)
# 行为者: Sarah Parke (weaver) and Mark Barrow (painter)
# 对象: Painting object representing "RUN" collaborative artwork
# 目的: To document multiple artists with different roles in production
artwork = vocab.Painting(ident="run/1", label="RUN")

# 步骤 4: 创建相关对象和关系
# Main production event
production = model.Production()
artwork.produced_by = production

# First artist contribution: Mark Barrow (painting)
painting_activity = model.Production()
painting_activity.technique = vocab.instances['painting']
painting_activity.carried_out_by = model.Person(ident="barrow", label="Mark Barrow")
production.part = painting_activity

# Second artist contribution: Sarah Parke (hand weaving)
weaving_activity = model.Production()
weaving_activity.technique = model.Type(
    ident="http://vocab.getty.edu/aat/300053643",
    label="hand weaving"
)
weaving_activity.carried_out_by = model.Person(ident="parke", label="Sarah Parke")
production.part = weaving_activity

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))



## 与创作相关的对象

其他对象在艺术品的创作中可能发挥关键作用，如复制其他艺术品或受其启发，或使用相同来源创作艺术品，无论是机械还是手工，如用于打印照片的底片或用于印刷蚀刻版的版材。

### 灵感、研究或复制品

一些艺术品是其他艺术品的复制品，或明显直接受其他作品启发。与另一作品的这种关系可以用 `Production` 活动的 `influenced_by` 属性来捕捉。复制品可能来自记忆或在复制品实物在场的情况下制作，可以是忠实的复制品或仅仅是可识别的相似。这包括为作品的最终版本所做的研究。模型中不捕捉影响的确切性质。

**示例**：

1964年，迪恩·凯勒创作了丹尼尔·亨廷顿1858年[詹姆斯·德怀特·达纳的肖像](https://artgallery.yale.edu/collections/objects/52687)的[复制品](https://artgallery.yale.edu/collections/objects/54339)。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (y of Huntington Portrait)
# 行为者: Deane Keller (the copier)
# 对象: Painting representing a copy of Daniel Huntington's portrait
# 目的: To document the relationship between the copy and the original artwork
copy_portrait = vocab.Painting(
    ident="kellerdana/1",
    label="Copy of Huntington Portrait"
)

# 步骤 4: 创建相关对象和关系
# Production event
production = model.Production()
copy_portrait.produced_by = production

# What influenced the production: Original Huntington Portrait
original_portrait = model.HumanMadeObject(
    ident="huntingtondana",
    label="Huntington Portrait of Dana"
)
production.influenced_by = original_portrait

# Who carried out the production: Deane Keller (the copier)
production.carried_out_by = model.Person(ident="keller", label="Deane Keller")

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))



### 从可识别来源的复制

许多对象直接从来源创作，如从底片打印的照片、从木刻版创作的版画，或从模具制作的雕塑。特定来源对象的使用可以用 `used_specific_object` 属性作为对象 `Production` 描述的一部分来捕捉。

请注意，从同一来源创作的所有艺术对象将显示相同的图像，无论是平面的还是三维的。来源也显示相同的图像，虽然可能以某种方式反转。图像被建模为所有物理对象显示的 `VisualItem`，允许我们将对象组合在一起。关于作品的更多信息，请参见[相关性](../aboutness/)部分。

设备或工具，如特定的相机或调色板，也将使用相同的属性 `used_specific_object` 建模，但是当然不会显示与主作品相同的视觉项目。

**示例**：

阿尔弗雷德·斯蒂格利茨拍摄的乔治亚·奥基夫的照片复制品，在不同时间从同一底片打印，现在由不同机构拥有：[耶鲁大学美术馆](https://artgallery.yale.edu/collections/objects/198690)、[国家美术馆](https://www.nga.gov/collection/art-object-page.60057.html)和[乔治亚·奥基夫博物馆](https://collections.okeeffemuseum.org/object/6627/)



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( 1918 photograph at Georgia O'Keeffe Museum)
# 行为者: Alfred Stieglitz (photographer), Georgia O'Keeffe Museum (owner)
# 对象: Photograph object representing a print from the same negative
# 目的: To document prints created from the same source negative
photo_gokm = vocab.Photograph(
    ident="okeeffe-gok/1",
    label="GOK 1918, GOKM"
)
photo_gokm.identified_by = vocab.AccessionNumber(content="2014.3.78")

# 步骤 4: 创建相关对象和关系
# Production event (printing from negative)
production = model.Production(label="Printing of Photograph")
photo_gokm.produced_by = production

# What specific object was used: The negative source
negative = model.HumanMadeObject(
    ident="okeeffe-negative",
    label="Negative of GOK 1918"
)
production.used_specific_object = negative

# What visual content is shown: The shared visual item
visual_content = model.VisualItem(
    ident="okeeffe",
    label="Visual Content of GOK 1918"
)
photo_gokm.shows = visual_content

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))

In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( 1918 photograph at Yale University Art Gallery)
# 行为者: Alfred Stieglitz (photographer), Yale University Art Gallery (owner)
# 对象: Another photograph print from the same negative
# 目的: To demonstrate how multiple prints from the same negative are documented
photo_yuag = vocab.Photograph(
    ident="okeeffe-yuag/1",
    label="GOK 1918, YUAG"
)
photo_yuag.identified_by = vocab.AccessionNumber(content="2016.101.242")

# 步骤 4: 创建相关对象和关系
# Production event (printing from negative)
production = model.Production(label="Printing of Photograph")
photo_yuag.produced_by = production

# What specific object was used: The same negative source
negative = model.HumanMadeObject(
    ident="okeeffe-negative",
    label="Negative of GOK 1918"
)
production.used_specific_object = negative

# What visual content is shown: The shared visual item
visual_content = model.VisualItem(
    ident="okeeffe",
    label="Visual Content of GOK 1918"
)
photo_yuag.shows = visual_content

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))



**示例**：

耶鲁大学英国艺术中心的一幅[版画](https://collections.britishart.yale.edu/catalog/tms:6141)，乔叟的《坎特伯雷朝圣者》，使用耶鲁大学美术馆持有的特定[铜版](https://artgallery.yale.edu/collections/objects/11787)制作。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (ucer's Canterbury Pilgrims print)
# 行为者: Unknown printmaker, Yale Center for British Art (owner)
# 对象: Print object made from a specific copper plate
# 目的: To document a print created from an identifiable source object
print_artwork = vocab.Print(
    ident="ccp/1",
    label="Chaucer's Canterbury Pilgrims"
)

# 步骤 4: 创建相关对象和关系
# Production event (printing from plate)
production = model.Production(label="Printing from Plate")
print_artwork.produced_by = production

# What specific object was used: The copper plate source
production.used_specific_object = model.HumanMadeObject(
    ident="ccp-plate",
    label="Plate for CCP"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))

In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 ( copper plate itself)
# 行为者: Unknown plate maker, Yale University Art Gallery (owner)
# 对象: HumanMadeObject representing the copper plate source
# 目的: To document the physical object (plate) used to create prints
copper_plate = model.HumanMadeObject(
    ident="ccp-plate/1",
    label="Plate for CCP"
)

# 步骤 4: 创建相关对象和关系
# What material the plate is made of: Copper
copper_plate.made_of = vocab.instances['copper']

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(copper_plate, compact=False))



## 创作原因

艺术品经常被委托制作，艺术家同意创作艺术品，委托人同意补偿艺术家的努力。[委托](/model/provenance/promises/#commissions-for-artwork)的建模在流传历史部分描述，作为承诺的交换，可能还包括付款。对象可以在其创作中使用 `caused_by` 属性引用此活动。

**示例**：

[Nuveen绘画](https://artgallery.yale.edu/collections/objects/290048)的创作是由其委托引起的。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (een Painting)
# 行为者: Jim Dine (artist), unknown commissioner
# 对象: HumanMadeObject representing "The Nuveen Painting"
# 目的: To document a painting caused by a commission agreement
nuveen_painting = model.HumanMadeObject(
    ident="nuveen/1",
    label="Nuveen Painting"
)
nuveen_painting.identified_by = vocab.PrimaryName(content="The Nuveen Painting")

# 步骤 4: 创建相关对象和关系
# Production event
production = model.Production()
nuveen_painting.produced_by = production

# Who carried out the production: Jim Dine (the artist)
production.carried_out_by = model.Person(ident="dine", label="Jim Dine")

# What caused the production: The commission activity
commission_activity = model.Activity(
    ident="nuveen_commission",
    label="Commission"
)
production.caused_by = commission_activity

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(nuveen_painting, compact=False))



## 未识别或未知艺术家

许多对象由未知或未识别的艺术家创作。虽然可以有一个名为"未识别艺术家"的不同人物记录，但这会很快在系统中创建大量已识别的人，每个只有一次引用。相反，建议创建一个群体记录来代表所有未识别的艺术家，然后每个对象都有其创作由该群体`carried_out_by`，意味着该人工群体集合中的一个（或多个）人。

如果知道艺术家的某些特征，例如他们的国籍或他们活跃的世纪，这些可以是具有这些属性的附加群体。

**示例**：

[Coppa Amatoria](https://artgallery.yale.edu/collections/objects/138452)由来自意大利的未识别艺术家或艺术家创作。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (pa Amatoria)
# 行为者: Unidentified Italian artist(s)
# 对象: HumanMadeObject representing "Coppa Amatoria"
# 目的: To document an artwork created by unidentified Italian artists
coppa = model.HumanMadeObject(
    ident="coppa/1",
    label="Coppa Amatoria"
)
coppa.identified_by = vocab.PrimaryName(content="Coppa Amatoria")

# 步骤 4: 创建相关对象和关系
# Production event
production = model.Production()
coppa.produced_by = production

# Who carried out the production: Unidentified Italian artists (represented as a Group)
production.carried_out_by = model.Group(
    ident="unknown_italian",
    label="Unidentified Italian"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(coppa, compact=False))




## 归属限定符

### 受艺术家影响

如果对象创作与某人之间存在某种联系，该人不是直接的艺术家但影响了创作，那么可以使用 `influenced_by` 属性引用该人。这些通常表达为"仿"、"以...风格"或"以...方式"的归属——它们通过将创作与直接影响它的人（可能是通过其作品体现，而不是通过个人联系）相关联来限定归属。

**示例**：

绘画《洗衣日》有意以温斯洛·霍默的方式创作。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (h Day painting)
# 行为者: Unknown artist, influenced by Winslow Homer
# 对象: Painting object representing "Wash Day"
# 目的: To document a painting created in the manner of another artist
wash_day = vocab.Painting(
    ident="washday/1",
    label="Wash Day"
)
wash_day.identified_by = vocab.PrimaryName(content="Wash Day")

# 步骤 4: 创建相关对象和关系
# Production event
production = model.Production()
wash_day.produced_by = production

# Who influenced the production: Winslow Homer (the influential artist)
production.influenced_by = model.Person(
    ident="whomer",
    label="Winslow Homer"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(production, compact=False))



### 与艺术家相关的群体归属

即使艺术家或艺术家群体的身份不完全清楚，这些人可能已知是某个群体的一部分，如更著名的"大师"的工作室。在这种情况下，代表工作室的`Group`可以是执行`Production`的行动者：这并不意味着群体的每个成员都参与了，只是其中至少一个参与了，就像说文档由组织编写或发布并不意味着所有员工都为该努力做出了贡献。

我们可以使用群体的`Formation`（创建）上的`influenced_by`属性将其与已知的人连接起来——在示例用例中是工作室的"大师"。"大师"可能没有参与群体，甚至在群体形成时还活着，因此不必然形成群体甚至是其成员。

这种方法可以用于工作室、画室、学生集合、追随者等。不是整个群体创作了对象，而是其中之一或多个创作了。

**示例**：

上述描述的"男子半身像"对象以前被认为由弗朗西斯·哈德伍德工作室创作，这是一个群体。下面的示例是该群体的记录。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (dio of Francis Harwood)
# 行为者: Francis Harwood (influential master), unknown studio members
# 对象: Group representing the studio of Francis Harwood
# 目的: To document a workshop/studio group influenced by a master artist
studio_group = vocab.Studio(
    ident="harwoodstudio/1",
    label="Studio of Francis Harwood"
)

# 步骤 4: 创建相关对象和关系
# Formation event of the studio group
formation = model.Formation()
studio_group.formed_by = formation

# Who influenced the formation: Francis Harwood (the master artist)
formation.influenced_by = model.Person(
    ident="http://vocab.getty.edu/ulan/500015886",
    label="Francis Harwood"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(formation, compact=False))



### 不确定或变化的归属

与历史艺术品相关的一条信息，随着研究和理解的改进而变化的是创作它的艺术家的身份。在`carried_out_by`中引用的行动者是当前的意见，但以前的归属仍然可以记录。用于此的模式在文档的[断言](/model/assertion/)部分有更详细的描述。


## 通过移除的创作

当一个对象从较大的对象中移除时，它可能进入文档存在状态。部分的制作只是整体制作的一部分，但在它被移除之前，它不需要单独的身份或存在。

这种情况相当频繁地发生，既有合法的动机，也有不诚实的动机。在保护工作中，通常需要在将方法应用于整体之前从对象中移除一个微小的碎片进行实验。如果实验有意想不到的副作用，那么整体对象被保存，而以几乎不可察觉的变化为代价。知道样品是在过去生产而现在被移除很重要，而不是在现在创作。

这种情况发生的第二种情景不幸地很常见。如果一个对象，如中世纪手稿，可以通过拆分成部分并单独销售每个部分来获得更高的利润，那么不诚实的卖家就会这样做。不是将一本无害的时祷书出售给单个买家，而是每个照明画都可以单独出售，然后剩余的载文页面要么被丢弃，要么随着时间的推移以大大降低的价格出售。手稿的这种分散今天仍在发生。

为了对此建模，不是使用`Production`，对象通过`PartRemoval`活动`removed_by`。该活动与所有其他活动一样，但它还有一个`diminished`属性，引用部分被移除的整体对象。如果知道来源对象被移除的信息，通常不会对部分有单独的`Production`。


**示例**：

收藏品是Libro dei Notai大师的一页渐进（手稿）页面。[来源](https://artgallery.yale.edu/collections/objects/51500)



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (e from a Gradual)
# 行为者: Master of the Libro dei Notai (creator), unknown agent who removed the page
# 对象: Page object removed from a larger manuscript
# 目的: To document an object that came into existence by being removed from a whole
page = vocab.Page(
    ident="gradualpage/1",
    label="Page from a Gradual"
)

# 步骤 4: 创建相关对象和关系
# PartRemoval event (when the page was removed from the manuscript)
removal = model.PartRemoval()

# What was diminished: The original gradual manuscript
whole_manuscript = model.HumanMadeObject(
    ident="gradual",
    label="Gradual"
)
removal.diminished = whole_manuscript

page.removed_by = removal

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(removal, compact=False))



## 发现与创作

如果对象通过被发现而不是被创作而进入有记载的历史，那么不是`Production`条目，而是`Encounter`。这适用于化石、宝石或其他自然历史物品，以及被创作但没有关于该创作的信息可用，或者对象的"发现"是物品流传历史的第一个重要条目。此模式可能仅用于对象的发现，而不是其他类型的相遇。

相遇也可以作为单独的事件记录，如[流传历史部分](../../provenance/encounters/)所述。单独的事件必须用于任何不是发现的相遇。

不使用`produced_by`，使用的属性是`encountered_by`，活动的类是`Encounter`。所有其他模式都是相同的，包括相遇发生的地点、谁遇到它、何时发生等等。角色模式也适用，相遇的`part`也是`Encounter`。

**示例**：

化石Torosaurus（VP.04072）于1891年被约翰·贝尔·哈彻发现。



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (osaurus fossil)
# 行为者: John Bell Hatcher (discoverer)
# 对象: HumanMadeObject representing the discovered fossil
# 目的: To document a natural history object that entered recorded history via discovery
torosaurus = model.HumanMadeObject(
    ident="torosaurus/1",
    label="Torosaurus Gladius"
)

# 步骤 4: 创建相关对象和关系
# Encounter event (discovery of the fossil)
encounter = model.Encounter()

# Who carried out the encounter: John Bell Hatcher (the discoverer)
encounter.carried_out_by = model.Person(
    ident="hatcher",
    label="John Bell Hatcher"
)

# When: The discovery occurred in 1891
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1891-01-01T00:00:00Z"
timespan.end_of_the_end = "1891-12-31T23:59:59Z"
encounter.timespan = timespan

torosaurus.encountered_by = encounter

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(torosaurus, compact=False))



## 毁坏

对象流传历史链的终点是已知被毁坏时。对象的丢失使链保持开放，因为它可能在将来被找回，但是如果对象被毁坏，那么就没有回头路。因此，对象只有在已知被毁坏时才应被记录为毁坏。

模型使用一个`Destruction`类，表示对象的不复存在。毁坏只适用于单个对象，因此每个对象都有自己的毁坏，即使该毁坏是由相同的更广泛事件或活动引起的。这意味着毁坏不是由行动者`carried_out_by`，而是（如下一节所述）另一个活动引起毁坏。

**示例**：

毕加索的绘画《Le Peintre》于1998年9月2日晚约10:30被毁坏（在飞机失事中）。[来源](https://en.wikipedia.org/wiki/Swissair_Flight_111)



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (Peintre by Picasso)
# 行为者: Pablo Picasso (artist), Swissair Flight 111 crash (destruction cause)
# 对象: Painting object that was destroyed
# 目的: To document the end of an object's provenance chain through destruction
painting = vocab.Painting(
    ident="lepeintre/1",
    label="Le Peintre by Picasso"
)

# 步骤 4: 创建相关对象和关系
# Destruction event
destruction = model.Destruction(label="Destruction of Le Peintre")
painting.destroyed_by = destruction

# When: The destruction occurred (September 2nd, 1998, approximately 10:30pm)
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1998-09-02T22:20:00Z"
timespan.end_of_the_end = "1998-09-02T22:40:00Z"
destruction.timespan = timespan

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(destruction, compact=False))



### 毁坏原因

如上所述，同一事件或活动可能是许多单个对象毁坏的原因。博物馆被烧毁或以其他方式被毁坏会毁坏许多对象，包括建筑物本身。在较小的规模上，个人毁坏可能是由个人的特定活动引起的。

为了区分毁坏本身及其原因，我们使用毁坏与引起它的活动或事件之间的`caused_by`关系。事件应该像模型中的任何其他事件一样被描述和分类。

**示例**：

Le Peintre因飞机坠毁而被毁坏。（这也可能导致飞机的毁坏、机组人员和乘客的死亡以及许多其他后果）



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (Peintre by Picasso)
# 行为者: Pablo Picasso (artist), Swissair Flight 111 crash (destruction cause)
# 对象: Painting object destroyed by a plane crash
# 目的: To document the cause of destruction separately from the destruction event itself
painting = vocab.Painting(
    ident="lepeintre/2",
    label="Le Peintre by Picasso"
)

# 步骤 4: 创建相关对象和关系
# Destruction event
destruction = model.Destruction(label="Destruction of Le Peintre")
painting.destroyed_by = destruction

# When: The destruction occurred
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1998-09-02T22:20:00Z"
timespan.end_of_the_end = "1998-09-02T22:40:00Z"
destruction.timespan = timespan

# What caused the destruction: The plane crash event
crash_event = model.Event(
    ident="sr111crash",
    label="Crash of Swiss Air 111"
)
destruction.caused_by = crash_event

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(destruction, compact=False))